# Explorative Analysis
I am using this notebook to explore the dataset of the Rust QSim and define some initial analysis figures.

In [10]:
%load_ext autoreload
%autoreload 2

import sys

print(sys.executable)
print(sys.path[:5])

# Importing necessary libraries
# import seaborn as sns

from pathlib import Path

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# import file load.py
from load import discover_runs, read_aggregated_instrument, read_aggregated_routing

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/opt/homebrew/anaconda3/envs/router/bin/python
['/Users/paulh/Applications/PyCharm.app/Contents/plugins/python-ce/helpers/jupyter_debug', '/Users/paulh/Applications/PyCharm.app/Contents/plugins/python-ce/helpers/pydev', '/opt/homebrew/anaconda3/envs/router/lib/python314.zip', '/opt/homebrew/anaconda3/envs/router/lib/python3.14', '/opt/homebrew/anaconda3/envs/router/lib/python3.14/lib-dynload']


## Load QSim Instrument Data

Now, let's discover the runs in the output directory and load them.

In [11]:
root = Path("/Users/paulh/hlrn-cluster/rust-pt-routing/parallel-qsim-berlin/output/v6.4")

runs = discover_runs(root)
print(runs)

x=4

[RunMeta(run_id='sim128_hor600_w4_r192_10pct', path=PosixPath('/Users/paulh/hlrn-cluster/rust-pt-routing/parallel-qsim-berlin/output/v6.4/sim128_hor600_w4_r192_10pct'), sim_cpus=128, horizon=600, worker_threads=4, router_threads=192, pct=10), RunMeta(run_id='sim128_hor600_w4_r192_1pct', path=PosixPath('/Users/paulh/hlrn-cluster/rust-pt-routing/parallel-qsim-berlin/output/v6.4/sim128_hor600_w4_r192_1pct'), sim_cpus=128, horizon=600, worker_threads=4, router_threads=192, pct=1), RunMeta(run_id='sim16_hor600_w4_r192_10pct', path=PosixPath('/Users/paulh/hlrn-cluster/rust-pt-routing/parallel-qsim-berlin/output/v6.4/sim16_hor600_w4_r192_10pct'), sim_cpus=16, horizon=600, worker_threads=4, router_threads=192, pct=10), RunMeta(run_id='sim16_hor600_w4_r192_1pct', path=PosixPath('/Users/paulh/hlrn-cluster/rust-pt-routing/parallel-qsim-berlin/output/v6.4/sim16_hor600_w4_r192_1pct'), sim_cpus=16, horizon=600, worker_threads=4, router_threads=192, pct=1), RunMeta(run_id='sim192_hor1800_w4_r192_10pc

In [4]:
qsim_data = {}
for run in runs:
    print(f"Loading run {run.run_id}")
    qsim_data[run.run_id] = read_aggregated_instrument(run)

Loading run sim128_hor600_w4_r192_10pct
Reading parquet:  /Users/paulh/hlrn-cluster/rust-pt-routing/parallel-qsim-berlin/output/v6.4/sim128_hor600_w4_r192_10pct/instrument/instrument_aggregated.parquet


KeyboardInterrupt: 

Extract some example run data from a dataset to validate the loading process.

In [ ]:
# use data from run "sim8_hor600_w4_r192_1pct". Filter for func_name=run
example_run_id = "sim8_hor600_w4_r192_1pct"
example_data = qsim_data[example_run_id]
example_data_run = example_data[example_data["func_name"] == "run"]
print(example_data_run.head())

del example_data, example_data_run

Prepare data for the main plot: RTR vs. Number of CPUs by Percentage

In [ ]:
# for all data, filter for func_name="run", group by run_id, take mean from duration_min_ns and add as "duration_ns" column, remain with sim_cpus, horizon, worker_threads, router_threads, pct
all_runs_data = pd.concat(qsim_data.values())
all_runs_data_run = all_runs_data[all_runs_data["func_name"] == "run"]
all_runs_grouped = all_runs_data_run.groupby("run_id").agg({"duration_min_ns": "mean"}).reset_index()
all_runs_grouped = all_runs_grouped.merge(
    all_runs_data_run[["run_id", "sim_cpus", "horizon", "worker_threads", "router_threads", "pct"]].drop_duplicates(),
    on="run_id",
)
all_runs_grouped = all_runs_grouped.rename(columns={"duration_min_ns": "duration_ns"})

# Add a new column "rtr"
all_runs_grouped["rtr"] = (60 * 60 * 36 * 1e9) / all_runs_grouped["duration_ns"]

print(all_runs_grouped.head())

In [ ]:
sns.set_theme(style="whitegrid")

plt.figure(figsize=(10, 6))

sns.lineplot(
    # filter all_runs_grouped for horizon=600, worker_threads=4, router_threads=192
    data=all_runs_grouped[
        (all_runs_grouped["horizon"] == 600) &
        (all_runs_grouped["worker_threads"] == 4) &
        (all_runs_grouped["router_threads"] == 192)
        ],
    x="sim_cpus",
    y="rtr",
    hue="pct",
    marker="o"
)

plt.title("RTR vs. Number of CPUs by Percentage")
plt.xlabel("Number of CPUs")
plt.ylabel("RTR")

# Log scales
plt.yscale("log", base=10)
plt.xscale("log", base=2)

ax = plt.gca()

from matplotlib.ticker import LogLocator, LogFormatter

ax.yaxis.set_major_locator(LogLocator(base=10.0))
ax.yaxis.set_major_formatter(LogFormatter(base=10))

ax.yaxis.set_minor_locator(LogLocator(base=10.0, subs=(0.5,)))

ax.grid(True, which="major", linewidth=1.0)
ax.grid(True, which="minor", linestyle=":", linewidth=0.6)

ax.set_ylim(top=10_000)

plt.legend(title="pct (%)")
plt.tight_layout()
plt.show()

Now, let's plot the stacked bar chart for phases.

In [ ]:
# print all unique func_names
print(all_runs_data["func_name"].unique())

# filter all runs data for func_name in ['move_links','move_nodes','notify_end_all', 'notify_wakeup_all']
# phases_of_interest = ['move_links', 'move_nodes', 'notify_end_all', 'notify_wakeup_all']
# phases_data = all_runs_data[all_runs_data["func_name"].isin(phases_of_interest)]
#
# # group by run_id and func_name. take mean of duration_mean_ns. Stick with horizon, worker_threads, router_threads, pct
# phases_grouped = phases_data.groupby(["run_id", "func_name"]).agg({"duration_mean_ns": "mean"}).reset_index()
# phases_grouped = phases_grouped.merge(
#     phases_data[["run_id", "sim_cpus", "horizon", "worker_threads", "router_threads", "pct"]].drop_duplicates(),
#     on="run_id",
# )
# print(phases_grouped.head())
#
# del phases_data

In [ ]:
do_step_data = all_runs_data[all_runs_data["func_name"] == "do_step"]

do_step_grouped = do_step_data.groupby(["run_id", "func_name"]).agg({"duration_mean_ns": "mean"}).reset_index()
do_step_grouped = do_step_grouped.merge(
    phases_data[["run_id", "sim_cpus", "horizon", "worker_threads", "router_threads", "pct"]].drop_duplicates(),
    on="run_id",
)

del do_step_data

In [ ]:
# plot bar chart. filter for func_name="do_step" with sim_cpus on x axis, duration_mean_ns on y axis
plt.figure(figsize=(10, 6))
sns.barplot(
    data=do_step_grouped,
    x="sim_cpus",
    y="duration_mean_ns",
    hue="pct"
)
plt.title("do_step Duration by Number of CPUs and Percentage")
plt.xlabel("Number of CPUs")
plt.ylabel("Duration Mean (ns)")
plt.legend(title="pct (%)")
plt.tight_layout()
plt.show()

In [ ]:
# plot stacked bar chart with sim_cpus on x axis, duration_mean_ns on y axis, hue func_name and stack, facet by pct
import numpy as np
import matplotlib.pyplot as plt

for pct in sorted(phases_grouped["pct"].unique()):

    data = phases_grouped[phases_grouped["pct"] == pct]

    pivot = (
        data.pivot(
            index="sim_cpus",
            columns="func_name",
            values="duration_mean_ns"
        )
        .sort_index()
    )

    # Treat x as categorical
    x_labels = pivot.index.astype(str)
    x_pos = np.arange(len(pivot))

    plt.figure(figsize=(12, 6))

    bottom = np.zeros(len(pivot))

    for func in pivot.columns:
        plt.bar(
            x_pos,
            pivot[func].values,
            bottom=bottom,
            label=func,
            width=0.7
        )
        bottom += pivot[func].values

    plt.title(f"Phase Durations by Number of CPUs ({pct}% sample)")
    plt.xlabel("Number of CPUs")
    plt.ylabel("Duration Mean (ns)")

    # Categorical x-axis
    plt.xticks(x_pos, x_labels)

    plt.legend(title="Phase")
    plt.tight_layout()
    plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

for pct in sorted(phases_grouped["pct"].unique()):

    data = phases_grouped[phases_grouped["pct"] == pct]

    pivot = (
        data.pivot(
            index="sim_cpus",
            columns="func_name",
            values="duration_mean_ns"
        )
        .sort_index()
    )

    # Normalize each row so the stacked bar sums to 1.0
    row_sums = pivot.sum(axis=1)
    pivot_rel = pivot.div(row_sums, axis=0).fillna(0.0)

    # Treat x as categorical
    x_labels = pivot_rel.index.astype(str)
    x_pos = np.arange(len(pivot_rel))

    plt.figure(figsize=(12, 6))

    bottom = np.zeros(len(pivot_rel))

    for func in pivot_rel.columns:
        vals = pivot_rel[func].values
        plt.bar(
            x_pos,
            vals,
            bottom=bottom,
            label=func,
            width=0.7
        )
        bottom += vals

    plt.title(f"Relative Phase Durations by Number of CPUs ({pct}% sample)")
    plt.xlabel("Number of CPUs")
    plt.ylabel("Relative duration (fraction of total)")

    plt.xticks(x_pos, x_labels)
    plt.ylim(0, 1)  # because proportions

    plt.legend(title="Phase")
    plt.tight_layout()
    plt.show()

## Loading Routing Data

In [ ]:
routing_data: dict[str, pd.DataFrame] = {}

for run in runs:
    print(f"Loading routing data for run {run.run_id}")
    routing_data[run.run_id] = read_aggregated_routing(run)


In [ ]:
ex_id = "sim4_hor600_w4_r192_10pct"

# group by rank and count number of occurrences of each rank
routing_example = routing_data[ex_id]

print(routing_example.head())
routing_example_grouped = routing_example.groupby("rank").size().reset_index(name="count")
print(routing_example_grouped.head())

del routing_example, routing_example_grouped

In [ ]:
all_routing_data = pd.concat(routing_data.values())

#filter out all replace_route entries where request_uuid_u128=0
all_routing_data = all_routing_data[
    ~((all_routing_data["func_name"] == "replace_route") & (all_routing_data["request_uuid_u128"] == 0))]

print(all_routing_data.head())

In [ ]:
# filter all_routing_data for pct=10 and func_name="replace_route". group by run_id and process_id, preserve sim_cpus,horizon,worker_threads,router_threads,pct. Sum duration_ns.
routing_filtered = all_routing_data[all_routing_data["func_name"] == "replace_route"]
routing_grouped = routing_filtered.groupby(["run_id", "process_id"]).agg({"duration_ns": "sum"}).reset_index()
routing_grouped = routing_grouped.merge(
    routing_filtered[["run_id", "sim_cpus", "horizon", "worker_threads", "router_threads", "pct"]].drop_duplicates(),
    on="run_id",
)
print(routing_grouped.head())

# join all_runs_data_run and routing_grouped on run_id and process_id to get total duration_ns
merged_routing = routing_grouped.merge(
    all_runs_data_run[["run_id", "process_id", "duration_min_ns"]],
    on=["run_id", "process_id"],
    suffixes=("_routing", "_total")
)
print(merged_routing.head())

del routing_filtered, routing_grouped

For each run, choose the process that needed to wait the longest for routing and plot that against the total duration_ns of that process.

In [ ]:
# plot bar chart. filter for pct=10. group by run_id, take max of duration_ns and duration_min_ns of the corresponding entry. Plot sim_cpus on x axis, duration_ns and duration_min_ns on y axis next to each other as bars. use seaborn
idx = merged_routing.groupby("run_id")["duration_ns"].idxmax()
max_routing_recv = merged_routing.loc[idx].reset_index(drop=True)
print(max_routing_recv.head())

for p in sorted(max_routing_recv["pct"].unique()):
    pct_ = max_routing_recv[max_routing_recv["pct"] == p]

    plt.figure(figsize=(10, 6))
    sns.pointplot(data=pct_,
                  x="sim_cpus",
                  y="duration_ns",
                  color="blue",
                  label="Cumulative Waiting for Routing (ns)",
                  marker="o")
    sns.pointplot(data=pct_,
                  x="sim_cpus",
                  y="duration_min_ns",
                  color="orange",
                  label="Total Duration (ns)",
                  marker="o")

    plt.title(f"Cumulative Waiting for Routing vs. Total Duration by Number of CPUs ({p}% sample)")
    plt.xlabel("Number of CPUs")
    plt.ylabel("Duration (ns)")
    plt.legend()
    plt.tight_layout()
    plt.show()